# Create Function Call Chains
This notebook creates the function call chains (the sequence of functions called by other functions) in NoLCAT based on the Function Call Chains diagram. **The notebook uses the commands for executing cells above and below that appear in the upper left menu of code cells; where to use them is marked in bold text.**

In [41]:
from collections import deque
import re

## Function Call Chain Finding Function
Function based on function at https://www.geeksforgeeks.org/find-paths-given-source-destination/#approach-2-using-bfs-o-2v-v-time-and-ov-space; which requires nodes to be assigned numbers.

In [42]:
def findFunctionCallChains(number_of_nodes, edges, start_node, end_node):
    graph = [[] for _ in range(number_of_nodes)]
    allCallChains = []
    q = deque()
    q.append([start_node])  # Initialize queue with the starting call chain

    # Build the graph from edges
    for edge in edges:
        graph[edge[0]].append(edge[1])
    
    while q:
        cc = q.popleft()
        current = cc[-1]

        # If destination is reached, store the call chain
        if current == end_node:
            allCallChains.append(cc)
        
        # Explore all adjacent vertices
        for adj in graph[current]:
            newCallChains = list(cc)
            newCallChains.append(adj)
            q.append(newCallChains)
    
    return allCallChains

## Function Call Chain Creation Procedure

### Get DOT File Contents as String

In [43]:
with open("function_call_chains.dot", encoding="utf-8") as f:
    DOT_file = f.read()

### Extract All Function Names
From a network graph perspective, the functions serve as nodes.

In [44]:
raw_node_names = re.findall(r'\s{8}(?:\w|_)+\s\[', DOT_file)
node_names = []
for name in raw_node_names:
    node_names.append(name[8:-2])
number_of_nodes = len(node_names)

### Extract All Calls
From a network graph perspective, the calls serve as edges.

In [45]:
raw_edge_names = re.findall(r'\s{4}(?:\w|_)+\s->\s(?:\w|_)+', DOT_file)
edge_names = []
for name in raw_edge_names:
    pair = name.split(" -> ")
    edge_names.append(
        (pair[0][4:], pair[1])
    )

### Assign Number Values to Functions
The function for finding call chains requires the nodes to be assigned sequential numbers starting with zero (0); since the numbers will need to be converted back to function names, a reverse lookup is also created. The assignment is done at this point so the number can be used to select the starting node.

In [46]:
lookup_by_name = dict()
n = 0

for k in node_names:
    lookup_by_name[k] = n
    n += 1

lookup_by_number = {v: k for k, v in lookup_by_name.items()}

### Create Edge Array with Numbers

In [47]:
edge_pairs = []
for pair in edge_names:
    edge_pairs.append(
        (lookup_by_name[pair[0]], lookup_by_name[pair[1]])
    )

### Get Possible Starting Functions
Functions that don't have a call chain (don't call any other NoLCAT functions) don't need to be listed as possible start nodes.

In [ ]:
possible_start_nodes = set()
for pair in edge_pairs:
    possible_start_nodes.add(pair[0])

for node in possible_start_nodes:
    print(f"{node}: {lookup_by_number[node]}")

### Select Call Chain Starting Function
**Perform "Execute Cells Above" on the cell below.** Using the list output above, select the starting function for the desired call chains; enter the corresponding number in the cell below, **then perform "Execute Cell and Below"** to finish the notebook.

In [49]:
start_node = 69

### Find Function Call Chains
Since the goal is to find all call chains, nut just those with a specific endpoint, the notebook looks for call chains between the designated starting function/node and all other functions/nodes.

In [50]:
possible_end_nodes = [k for k in lookup_by_number.keys() if k != start_node]
function_call_chains = []

for end_node in possible_end_nodes:
    call_chains = findFunctionCallChains(number_of_nodes, edge_pairs, start_node, end_node)
    for call_chain in call_chains:
        if call_chain:
            function_call_chains.append(call_chain)

### Output Function Call Chains

In [ ]:
for call_chain in function_call_chains:
    temp = []
    for node in call_chain:
        temp.append(lookup_by_number[node])
    print(" -> ".join(temp))